# How to evaluate a RAG application

This example uses [Langchain](https://www.langchain.com) and [Giskard](https://github.com/Giskard-AI/giskard) to evaluate the quality of a RAG application.

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"

## Scrape the Website and Split the Content

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

loader = WebBaseLoader("https://abduls-organization-13.gitbook.io/abduls-portfolio-demo-1#my-skills-1")
documents = loader.load_and_split(text_splitter)
documents

[Document(page_content="About Me | Abdul's Portfolio DemoAbdul's Portfolio DemoSearchCtrl‚ÄÜ+‚ÄÜK\uf8ffüë®‚Äç\uf8ffüíªAbout MeProjects\uf8ffüíºJobInsights: Streamlining Your Job Hunt with AI\uf8ffüåêWebChatAI\uf8ffü´ÄChest-Cancer-Classification-Using-mlflow-and-DVC\uf8ffüìâCSVAnalystAI- Your AI Data analyst\uf8ffüìÑResume Screening Assistance\uf8ffüìúScriptMaster AI„ÄΩÔ∏èMini Projects Repository \uf8ffüöÄAbout us\uf8ffüë®‚Äç\uf8ffüíªMore About MeVisionSkillsCertificationsthank\uf8ffü§ùThank You for Visiting My Portfolio!Powered by GitBook\uf8ffüë®‚Äç\uf8ffüíªAbout MeWHO AM I !Professional SummaryAbdul Samad is Self-taught Machine Learning Engineer with a strong passion for developing software using a diverse range of ML and non-ML tools and APIs. Proficient in Python, Machine Learning, Deep Learning, NLP, computer vision, and generative AI, demonstrated through extensive project experience. Experienced with LLM libraries such as Langchain and Llama-index, as well as proficient in worki

## Load the Content in a Vector Store

In [10]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    documents, embedding=OpenAIEmbeddings()
)

## Create a Knowledge Base

Let's start by loading the content in a pandas DataFrame.

In [11]:
import pandas as pd

df = pd.DataFrame([d.page_content for d in documents], columns=["text"])
df.head(10)

,text
0,About Me | Abdul's Portfolio DemoAbdul's Portf...
1,MLOps and LLMOps. Committed to utilizing AI fo...
2,"My skills‚Ä¢ Programming LanguagesPython, C , ..."
3,Streamlining Your Job Hunt with AIIn today's c...
4,to offer various features and strategies to se...
5,reach out to potential employers.Future Develo...
6,feel free to open a pull request or an issue o...
7,"web scraping.The ChallengeLLMs, while powerful..."
8,related to the query.Response Generation: Base...
9,or finding local businesses.ConclusionWebChatA...


We can now create a Knowledge Base using the DataFrame we created before.

In [15]:
from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(df)
knowledge_base

## Generate the Test Set

In [16]:
from giskard.rag import generate_testset

testset = generate_testset(
    knowledge_base,
    num_questions=10,
    agent_description="A chatbot answering questions about Abdul Samad from his portfolio Website",
)

INFO:giskard.rag:Finding topics in the knowledge base.
c:\Users\abdulsamad\anaconda3\envs\evaluation\Lib\site-packages\umap\umap_.py:2437: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(
INFO:giskard.rag:Found 3 topics in the knowledge base.
Generating questions: 100%|██████████| 10/10 [01:12<00:00,  7.29s/it]


Let's display a few samples from the test set.

In [17]:
test_set_df = testset.to_pandas()

for index, row in enumerate(test_set_df.head(3).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")


Question 1: What is the aim of the Chest-Cancer-Classification-Using-mlflow-and-DVC project?
Reference answer: The project focuses on developing a deep learning model for predicting breast cancer risk and subtype, specifically adenocarcinoma, using chest CT scan images. This model aims to improve early detection and diagnosis, leading to more personalized treatment strategies.
Reference context:
Document 9: or finding local businesses.ConclusionWebChatAI's ability to scrape the web for real-time information sets it apart from traditional LLMs. By combining the power of AI with web scraping, WebChatAI offers a more dynamic and accurate conversational experience, making it a valuable tool for businesses and individuals alike.projects 3 ---------------------------------------------------Tutorial will come soon...Chest-Cancer-Classification-Using-mlflow-and-DVCIntroductionThe project focuses on developing a deep learning model for predicting breast cancer risk and subtype, specifically ade

Let's now save the test set to a file:

In [18]:
testset.save("test-set.jsonl")

## Prepare the Prompt Template

In [27]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



## Create the RAG Chain

Create a retriever from the Vector Store that will allow us to get the top similar documents to a given question.

In [21]:
retriever = vectorstore.as_retriever()

In [23]:
# dir(retriever)

In [25]:

retriever.invoke("Who is Abdul ?")

[Document(page_content="About Me | Abdul's Portfolio DemoAbdul's Portfolio DemoSearchCtrl‚ÄÜ+‚ÄÜK\uf8ffüë®‚Äç\uf8ffüíªAbout MeProjects\uf8ffüíºJobInsights: Streamlining Your Job Hunt with AI\uf8ffüåêWebChatAI\uf8ffü´ÄChest-Cancer-Classification-Using-mlflow-and-DVC\uf8ffüìâCSVAnalystAI- Your AI Data analyst\uf8ffüìÑResume Screening Assistance\uf8ffüìúScriptMaster AI„ÄΩÔ∏èMini Projects Repository \uf8ffüöÄAbout us\uf8ffüë®‚Äç\uf8ffüíªMore About MeVisionSkillsCertificationsthank\uf8ffü§ùThank You for Visiting My Portfolio!Powered by GitBook\uf8ffüë®‚Äç\uf8ffüíªAbout MeWHO AM I !Professional SummaryAbdul Samad is Self-taught Machine Learning Engineer with a strong passion for developing software using a diverse range of ML and non-ML tools and APIs. Proficient in Python, Machine Learning, Deep Learning, NLP, computer vision, and generative AI, demonstrated through extensive project experience. Experienced with LLM libraries such as Langchain and Llama-index, as well as proficient in worki

We can now create our chain.

In [28]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

Let's make sure the chain works by testing it with a simple question.

In [29]:
chain.invoke({"question": "who is Abdul ?"})

'Abdul is a self-taught Machine Learning Engineer with a strong passion for developing software using a diverse range of ML and non-ML tools and APIs. He is proficient in Python, Machine Learning, Deep Learning, NLP, computer vision, and generative AI, as demonstrated through extensive project experience.'

## Evaluating the Model on the Test Set

We need to create a function that invokes the chain with a specific question and returns the answer.

In [30]:
def answer_fn(question, history=None):
    return chain.invoke({"question": question})

We can now use the `evaluate()` function to evaluate the model on the test set. This function will compare the answers from the chain with the reference answers in the test set.

In [31]:
from giskard.rag import evaluate

report = evaluate(answer_fn, testset=testset, knowledge_base=knowledge_base)

CorrectnessMetric evaluation: 100%|██████████| 10/10 [00:23<00:00,  2.33s/it]


Let now display the report.

Here are the five components of our RAG application:

* **Generator**: This is the LLM used in the chain to generate the answers.
* **Retriever**: This is the retriever that fetches relevant documents from the knowledge base according to a query.
* **Rewriter**: This is a component that rewrites the user query to make it more relevant to the knowledge base or to account for chat history.
* **Router**: This is a component that filters the query of the user based on his intentions.
* **Knowledge Base**: This is the set of documents given to the RAG to generate the answers.

In [32]:
display(report)

Loading BokehJS ...

In [33]:
report.to_html("report.html")

We can display the correctness results organized by question type.

In [34]:
report.correctness_by_question_type()

,correctness
question_type,
complex,1.0
conversational,0.0
distracting element,0.0
double,0.0
simple,1.0
situational,1.0


We can also display the specific failures.

In [52]:
report.get_failures()

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
964fc6fa-491d-4b6a-aa9d-835328b2443e,What does the Machine Learning Systems course ...,The Machine Learning Systems course offers 18 ...,Document 0: Building Machine Learning Systems ...,[],"{'question_type': 'simple', 'seed_document_id'...",The Machine Learning Systems course offers 10 ...,False,The agent's answer includes details that are n...
7fb7d403-67a1-429b-bc19-d5d014074a0f,What does the Machine Learning Systems course ...,The Machine Learning Systems course offers 18 ...,Document 0: Building Machine Learning Systems ...,[],"{'question_type': 'simple', 'seed_document_id'...",The Machine Learning Systems course offers pra...,False,The agent's answer includes details that are n...
9cf283f9-f19b-4c58-b27b-fca0c5cc7ab2,What is the cost for joining the Machine Learn...,The cost for joining the Machine Learning prog...,Document 9: pay once to join the program and g...,[],"{'question_type': 'simple', 'seed_document_id'...",The cost for joining the Machine Learning prog...,False,The agent's answer is incorrect because it doe...
9f9fd3a9-0b3c-48fa-a90b-bd970238cd1e,What resources and benefits can I expect to re...,"When you join, you get lifetime access to 18 h...",Document 0: Building Machine Learning Systems ...,[],"{'question_type': 'complex', 'seed_document_id...",Upon enrolling in the Machine Learning Systems...,False,The agent's answer does not match the ground t...
8881989e-d95f-462f-931d-8604a75139f7,Could you provide information about the instru...,The instructor of the program is Santiago. He ...,Document 9: pay once to join the program and g...,[],"{'question_type': 'distracting element', 'seed...",The instructor of the Machine Learning program...,False,The agent's answer is partially correct but it...
cf85c206-d8e7-465d-a2a1-040f91ac49aa,What is the cost of the program that includes ...,The cost of the program is $450. It includes l...,Document 9: pay once to join the program and g...,[],"{'question_type': 'distracting element', 'seed...",The cost of the program is a one-time payment ...,False,The agent did not provide the specific cost of...
a805b425-f0b8-480a-9bb7-bbceb19263d2,Considering the course 'Building Machine Learn...,The cost of the program is $450. This includes...,Document 5: work.Wednesday: Optional office ho...,[],"{'question_type': 'distracting element', 'seed...",The course 'Building Machine Learning Systems ...,False,The agent did not provide the correct cost of ...
b07946fd-28af-42d6-9117-8959af8b1d9d,What is the cost to join the program that incl...,The cost to join the program is $450. It inclu...,Document 9: pay once to join the program and g...,[],"{'question_type': 'distracting element', 'seed...",The cost to join the program that includes des...,False,The agent did not provide the specific cost of...
d3af5418-5b97-4397-8173-d7238e219adf,"Considering the program's time commitment, wha...",The second session of the course covers topics...,Document 7: labels and weak supervision.Active...,[],"{'question_type': 'distracting element', 'seed...",For those interested in implementing the codin...,False,The agent's answer does not match the ground t...


## Creating a Test Suite

We can create a test suite and use it to compare different models.

Load the test set from disk.

In [35]:
from giskard.rag import QATestset

testset = QATestset.load("test-set.jsonl")

Create a Test Suite from the test set.

In [36]:
test_suite = testset.to_test_suite("Machine Learning School Test Suite")

We need a function that takes a DataFrame of questions, invokes the chain with each question, and returns the answers.

In [37]:
import giskard


def batch_prediction_fn(df: pd.DataFrame):
    return chain.batch([{"question": q} for q in df["question"].values])

We can now create a Giskard Model object to run our test suite.

In [38]:
giskard_model = giskard.Model(
    model=batch_prediction_fn,
    model_type="text_generation",
    name="Machine Learning School Question and Answer Model",
    description="This model answers questions about the Machine Learning School website.",
    feature_names=["question"], 
)

INFO:giskard.models.automodel:Your 'prediction_function' is successfully wrapped by Giskard's 'PredictionFunctionModel' wrapper class.


Let's now run the test suite using the model we created before.

In [39]:
test_suite_results = test_suite.run(model=giskard_model)

INFO:giskard.datasets.base:Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
INFO:giskard.utils.logging_utils:Predicted dataset with shape (10, 5) executed in 0:00:03.577083
ERROR:root:An error happened during test execution for test: TestsetCorrectnessTest
Traceback (most recent call last):
  File "c:\Users\abdulsamad\anaconda3\envs\evaluation\Lib\site-packages\giskard\core\suite.py", line 573, in run
    result = test_partial.giskard_test(**test_params).execute()
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\abdulsamad\anaconda3\envs\evaluation\Lib\site-packages\giskard\registry\giskard_test.py", line 192, in execute
    return configured_validate_arguments(self.test_fn)(*self.args, **self.kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pydantic\decorator.py", line 40, in pydantic.decorator.validate_arguments.validate.wrapper_function
  File "pydantic\decorator.py", line

We can display the results.

In [40]:
display(test_suite_results)

<TestSuiteResult (failed)>

## Integrating with Pytest

In [41]:
import ipytest

We can now integrate our test suite with Pytest.

In [42]:
%%ipytest

import pytest
from giskard.rag import QATestset
from giskard.testing.tests.llm import test_llm_correctness


@pytest.fixture
def dataset():
    testset = QATestset.load("test-set.jsonl")
    return testset.to_dataset()


@pytest.fixture
def model():
    return giskard_model


def test_chain(dataset, model):
    test_llm_correctness(model=model, dataset=dataset, threshold=0.5).assert_()

UsageError: Cell magic `%%ipytest` not found.
